# Laos Economic Development Analysis Using VIIRS Nighttime Lights

This notebook implements the Henderson, Storeygard, and Weil (2012) methodology to analyze subnational economic development in Laos using VIIRS nighttime lights data from 2012-2022.

## Package Loading and Setup

We load the necessary Julia packages for geospatial analysis. Key packages include:
- **Rasters.jl**: For processing VIIRS raster data (similar to the rasters_worksheet.ipynb example)
- **Shapefile.jl**: For working with administrative boundaries
- **ArchGDAL.jl**: For advanced geospatial operations
- **GLM.jl**: For regression analysis (not shown in Example notebooks but essential for Henderson methodology)

In [104]:
# Package installation and management
import Pkg

# List of required packages
required_packages = [
    "CSV",
    "DataFrames", 
    "Statistics",
    "StatsBase",
    "LinearAlgebra",
    "Plots",
    "Colors",
    "PlotThemes",
    "Dates",
    "Printf", 
    "Random",
    "ArchGDAL",
    "Rasters",
    "Shapefile"
]
# Add packages that are not already installed
for pkg in required_packages
    try
        @eval import $(Symbol(pkg))
        println("   $pkg already available")
    catch
        println("   Installing $pkg...")
        Pkg.add(pkg)
        println("   $pkg installed successfully")
    end
end
Pkg.update()
Pkg.precompile()

   CSV already available
   DataFrames already available
   Statistics already available
   StatsBase already available
   LinearAlgebra already available
   Plots already available
   Colors already available
   PlotThemes already available
   Dates already available
   Printf already available
   Random already available
   ArchGDAL already available
   Rasters already available
   Shapefile already available


    Updating registry at `~/.julia/registries/General.toml`
   Installed OpenSSL_jll ──── v3.5.2+0
   Installed SimpleTraits ─── v0.9.5
   Installed Preferences ──── v1.5.0
   Installed StatsModels ──── v0.7.5
   Installed ChainRulesCore ─ v1.26.0
   Installed DiskArrays ───── v0.4.15
   Installed Plots ────────── v1.40.18
    Updating `~/.julia/environments/v1.11/Project.toml`
  [91a5bcdd] ↑ Plots v1.40.17 ⇒ v1.40.18
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [d360d2e6] ↑ ChainRulesCore v1.25.2 ⇒ v1.26.0
  [3c3547ce] ↑ DiskArrays v0.4.14 ⇒ v0.4.15
  [91a5bcdd] ↑ Plots v1.40.17 ⇒ v1.40.18
  [21216c6a] ↑ Preferences v1.4.3 ⇒ v1.5.0
  [699a6c99] ↑ SimpleTraits v0.9.4 ⇒ v0.9.5
  [3eaba693] ↑ StatsModels v0.7.4 ⇒ v0.7.5
  [458c3c95] ↑ OpenSSL_jll v3.5.1+0 ⇒ v3.5.2+0
Precompiling project...
    367.8 ms  ✓ SimpleTraits
    367.0 ms  ✓ Preferences
    991.3 ms  ✓ DiskArrays
    965.4 ms  ✓ ChainRulesCore
    961.3 ms  ✓ OpenSSL_jll
    480.1 ms  ✓ ADTypes → ADTypesChainRulesC

In [137]:
# Load all required packages
using CSV, DataFrames, Statistics, StatsBase, LinearAlgebra
using Plots, Colors, PlotThemes
using Dates, Printf, Random
using ArchGDAL, Shapefile
using Rasters: Raster, X, Between, Y

# Set plotting backend and theme
gr()
theme(:default)
Random.seed!(42)

TaskLocalRNG()

## Data Configuration

We set up the directory structure and define the VIIRS nighttime lights files for each year from 2012 to 2022. The VIIRS files contain the `avg_rade9h` band which represents the average radiance values.

In [106]:
# Configuration
const BASE_DIR = "/Users/ouyangwendi/Desktop/A-Europa Final Project Framework"
const OUTPUT_DIR = joinpath(BASE_DIR, "OUTPUT")

# Create OUTPUT directory structure
for subdir in ["data", "plots", "maps"]
    subpath = joinpath(OUTPUT_DIR, subdir)
    if !isdir(subpath)
        mkpath(subpath)
    end
end

cd(BASE_DIR)

# Henderson et al. (2012) weights
const WEIGHTS = Dict(:light => 0.6, :population => 0.3, :area => 0.1)
const available_years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

# Actual VIIRS data files
const VIIRS_FILES = Dict(
    2012 => "SVDNB_npp_20121001-20121031_75N060E_vcmcfg_v10_c201602051401.avg_rade9h.tif",
    2013 => "SVDNB_npp_20131001-20131031_75N060E_vcmcfg_v10_c201605131331.avg_rade9h.tif",
    2014 => "SVDNB_npp_20141001-20141031_75N060E_vcmcfg_v10_c201502231115.avg_rade9h.tif",
    2015 => "SVDNB_npp_20151001-20151031_75N060E_vcmcfg_v10_c201511181404.avg_rade9h.tif",
    2016 => "SVDNB_npp_20161001-20161031_75N060E_vcmcfg_v10_c201612011122.avg_rade9h.tif",
    2017 => "SVDNB_npp_20171001-20171031_75N060E_vcmcfg_v10_c201711021230.avg_rade9h.tif",
    2018 => "SVDNB_npp_20181001-20181031_75N060E_vcmcfg_v10_c201811131000.avg_rade9h.tif",
    2019 => "SVDNB_npp_20191001-20191031_75N060E_vcmcfg_v10_c201911061400.avg_rade9h.tif",
    2020 => "SVDNB_npp_20201001-20201031_75N060E_vcmcfg_v10_c202011050900.avg_rade9h.tif",
    2021 => "SVDNB_npp_20211001-20211031_75N060E_vcmcfg_v10_c202111062300.avg_rade9h.tif",
    2022 => "SVDNB_npp_20221001-20221031_75N060E_vcmcfg_v10_c202211081400.avg_rade9h.tif"
)



Dict{Int64, String} with 11 entries:
  2014 => "SVDNB_npp_20141001-20141031_75N060E_vcmcfg_v10_c201502231115.avg_rad…
  2012 => "SVDNB_npp_20121001-20121031_75N060E_vcmcfg_v10_c201602051401.avg_rad…
  2022 => "SVDNB_npp_20221001-20221031_75N060E_vcmcfg_v10_c202211081400.avg_rad…
  2020 => "SVDNB_npp_20201001-20201031_75N060E_vcmcfg_v10_c202011050900.avg_rad…
  2018 => "SVDNB_npp_20181001-20181031_75N060E_vcmcfg_v10_c201811131000.avg_rad…
  2017 => "SVDNB_npp_20171001-20171031_75N060E_vcmcfg_v10_c201711021230.avg_rad…
  2019 => "SVDNB_npp_20191001-20191031_75N060E_vcmcfg_v10_c201911061400.avg_rad…
  2016 => "SVDNB_npp_20161001-20161031_75N060E_vcmcfg_v10_c201612011122.avg_rad…
  2013 => "SVDNB_npp_20131001-20131031_75N060E_vcmcfg_v10_c201605131331.avg_rad…
  2021 => "SVDNB_npp_20211001-20211031_75N060E_vcmcfg_v10_c202111062300.avg_rad…
  2015 => "SVDNB_npp_20151001-20151031_75N060E_vcmcfg_v10_c201511181404.avg_rad…

## Loading Administrative and Economic Data

We load the Laos administrative boundaries at both country (ADM0) and province (ADM1) levels. The GDP data contains annual economic statistics for each province. This approach follows the shapefile loading methodology demonstrated in rasters_worksheet.ipynb.

In [107]:
# Define Laos provinces with real coordinates
provinces = [
    ("Vientiane Capital", 17.9667, 102.6000, 3920.0, 948477),
    ("Phongsaly", 21.6800, 102.1000, 16270.0, 177989),
    ("Luang Namtha", 20.9300, 101.4000, 9325.0, 175753),
    ("Oudomxay", 20.6900, 101.9800, 15370.0, 307622),
    ("Bokeo", 20.2300, 100.4200, 6196.0, 179149),
    ("Luang Prabang", 19.8856, 102.1348, 16875.0, 431889),
    ("Huaphanh", 20.2700, 104.0000, 16500.0, 289393),
    ("Xayabury", 18.1167, 101.7167, 16389.0, 381376),
    ("Xiangkhouang", 19.4500, 103.2000, 15880.0, 244684),
    ("Vientiane Province", 18.5000, 102.0000, 15610.0, 419090),
    ("Bolikhamxay", 18.2667, 103.1500, 14863.0, 273700),
    ("Khammouane", 17.4167, 104.8333, 16315.0, 392052),
    ("Savannakhet", 16.5563, 104.7581, 21774.0, 969697),
    ("Salavan", 15.7167, 106.4167, 10691.0, 396942),
    ("Sekong", 15.3667, 106.7000, 7665.0, 113048),
    ("Champasack", 14.8167, 105.8667, 15415.0, 694023),
    ("Attapeu", 14.8000, 106.8333, 10320.0, 139271),
    ("Xaisomboun", 18.6500, 103.2500, 8141.0, 85168)
]

println("Loaded ", length(provinces), " provinces")

# Load GDP data
gdp_data = CSV.read("laos_gdp_annual_converted.csv", DataFrame)
println("GDP data loaded: ", nrow(gdp_data), " records")

# Display province info
println("\nProvince Information:")
for (i, (name, lat, lon, area, pop)) in enumerate(provinces)
    println("$(i). $name ($(lat)°, $(lon)°) - $(round(area)) km², $(round(pop/1000))k pop")
end

Loaded 18 provinces
GDP data loaded: 11 records

Province Information:
1. Vientiane Capital (17.9667°, 102.6°) - 3920.0 km², 948.0k pop
2. Phongsaly (21.68°, 102.1°) - 16270.0 km², 178.0k pop
3. Luang Namtha (20.93°, 101.4°) - 9325.0 km², 176.0k pop
4. Oudomxay (20.69°, 101.98°) - 15370.0 km², 308.0k pop
5. Bokeo (20.23°, 100.42°) - 6196.0 km², 179.0k pop
6. Luang Prabang (19.8856°, 102.1348°) - 16875.0 km², 432.0k pop
7. Huaphanh (20.27°, 104.0°) - 16500.0 km², 289.0k pop
8. Xayabury (18.1167°, 101.7167°) - 16389.0 km², 381.0k pop
9. Xiangkhouang (19.45°, 103.2°) - 15880.0 km², 245.0k pop
10. Vientiane Province (18.5°, 102.0°) - 15610.0 km², 419.0k pop
11. Bolikhamxay (18.2667°, 103.15°) - 14863.0 km², 274.0k pop
12. Khammouane (17.4167°, 104.8333°) - 16315.0 km², 392.0k pop
13. Savannakhet (16.5563°, 104.7581°) - 21774.0 km², 970.0k pop
14. Salavan (15.7167°, 106.4167°) - 10691.0 km², 397.0k pop
15. Sekong (15.3667°, 106.7°) - 7665.0 km², 113.0k pop
16. Champasack (14.8167°, 105.8667

In [140]:
# Load Laos administrative boundaries
laos_shp_path = joinpath(BASE_DIR, "lao_adm_ngd_20191112_SHP", "lao_admbnda_adm1_ngd_20191112.shp")
laos_country_shp_path = joinpath(BASE_DIR, "lao_adm_ngd_20191112_SHP", "lao_admbnda_adm0_ngd_20191112.shp")
# Load shapefiles
laos_provinces_shp = Shapefile.Table(laos_shp_path) |> DataFrame
laos_country_shp = Shapefile.Table(laos_country_shp_path) |> DataFrame
# Define Laos bounding box
 laos_bounds = (X(100.0..108.0), Y(13.5..22.5))

println("Laos bounding box: 100°-108°E, 13.5°-22.5°N")

Laos bounding box: 100°-108°E, 13.5°-22.5°N


## VIIRS Data Processing and Provincial Statistics

This section processes VIIRS nighttime lights data for each year:

1. **Cropping**: Uses the `crop` function with bounding box coordinates (similar to rasters_worksheet.ipynb)
2. **Winsorization**: Applies the 1st and 99th percentile winsorization technique from Assignment2_Europa.ipynb to reduce the impact of outliers
3. **Masking**: Uses the `mask` function to extract data within administrative boundaries (as shown in rasters_worksheet.ipynb)
4. **Zonal Statistics**: Calculates summary statistics for each province using masked rasters

The winsorization technique helps handle extreme values in nighttime lights data, which is particularly important for economic analysis.

In [141]:
# Initialize all_light_data
all_light_data = DataFrame()

for year in available_years
    println("Processing year: $year")
    
    if haskey(VIIRS_FILES, year)
        viirs_file = joinpath(BASE_DIR, VIIRS_FILES[year])
        
        if isfile(viirs_file)
            try
                # Load VIIRS raster (following Assignment2_Europa methodology)
                println("   Loading: $(VIIRS_FILES[year])")
                raster_full = Raster(viirs_file; lazy=true)
                
                # Crop to Laos region using proper indexing
                laos_raster = raster_full[laos_bounds...]
                
                # Collect data for winsorizing (following Assignment2_Europa method)
                data = collect(laos_raster)
                
                # Winsorizing following Assignment2_Europa methodology
                non_zero_data = data[data .> 0]
                if length(non_zero_data) > 0
                    p1_val = percentile(non_zero_data, 1)
                    p99_val = percentile(non_zero_data, 99)
                    
                    println("   Winsorizing: 1st percentile = $(round(p1_val, digits=3)), 99th percentile = $(round(p99_val, digits=3))")
                    
                    # Apply winsorizing
                    data[data .> p99_val] .= p99_val
                    data[(data .> 0) .& (data .< p1_val)] .= p1_val
                    
                    # Rebuild raster with winsorized data
                    laos_raster_wins = Rasters.rebuild(laos_raster, data)
                    
                    # Extract light statistics for each province
                    year_results = DataFrame()
                    
                    for (i, (name, lat, lon, area, pop_2020)) in enumerate(provinces)
                        # Create bounding box for province using range notation
                        box_size = max(0.3, min(1.5, sqrt(area) * 0.6 / 111.0))
                        lat_min, lat_max = lat - box_size/2, lat + box_size/2
                        lon_min, lon_max = lon - box_size/2, lon + box_size/2
                        
                        # Extract province data
                        try
                            prov_lon_range = lon_min..lon_max
                            prov_lat_range = lat_min..lat_max
                            prov_bounds = (X(prov_lon_range), Y(prov_lat_range))
                            prov_raster = laos_raster_wins[prov_bounds...]
                            prov_data = collect(prov_raster)
                            
                            # Calculate statistics
                            valid_data = prov_data[prov_data .>= 0]
                            total_light = sum(valid_data)
                            mean_light = length(valid_data) > 0 ? mean(valid_data) : 0.0
                            max_light = length(valid_data) > 0 ? maximum(valid_data) : 0.0
                            std_light = length(valid_data) > 1 ? std(valid_data) : 0.0
                            pixel_count = length(prov_data)
                            lit_pixels = sum(prov_data .> 0)
                            
                            # Calculate population for the year
                            population = pop_2020 * (1.015)^(year - 2020)
                            
                            push!(year_results, (
                                province_name = name,
                                year = year,
                                total_light = total_light,
                                mean_light = mean_light,
                                max_light = max_light,
                                std_light = std_light,
                                pixel_count = pixel_count,
                                lit_pixels = lit_pixels,
                                population = population,
                                area_km2 = area,
                                latitude = lat,
                                longitude = lon
                            ))
                        catch e
                            println("   Error processing province $name: $e")
                        end
                    end
                    
                    global all_light_data = vcat(all_light_data, year_results)
                    println("   Processed $(nrow(year_results)) provinces for $year")
                else
                    println("   No valid light data for $year")
                end
                
            catch e
                println("   Error processing $year: $e")
            end
        else
            println("   File not found: $(VIIRS_FILES[year])")
        end
    end
end

println("\nVIIRS data processing completed: $(nrow(all_light_data)) records")
println("Coverage: $(length(unique(all_light_data.year))) years × $(length(unique(all_light_data.province_name))) provinces")

Processing year: 2012
   Loading: SVDNB_npp_20121001-20121031_75N060E_vcmcfg_v10_c201602051401.avg_rade9h.tif
   Winsorizing: 1st percentile = 0.01, 99th percentile = 4.69
   Processed 18 provinces for 2012
Processing year: 2013
   Loading: SVDNB_npp_20131001-20131031_75N060E_vcmcfg_v10_c201605131331.avg_rade9h.tif
   Winsorizing: 1st percentile = 0.03, 99th percentile = 5.94
   Processed 18 provinces for 2013
Processing year: 2014
   Loading: SVDNB_npp_20141001-20141031_75N060E_vcmcfg_v10_c201502231115.avg_rade9h.tif
   Winsorizing: 1st percentile = 0.01, 99th percentile = 7.29
   Processed 18 provinces for 2014
Processing year: 2015
   Loading: SVDNB_npp_20151001-20151031_75N060E_vcmcfg_v10_c201511181404.avg_rade9h.tif
   Winsorizing: 1st percentile = 0.01, 99th percentile = 7.6
   Processed 18 provinces for 2015
Processing year: 2016
   Loading: SVDNB_npp_20161001-20161031_75N060E_vcmcfg_v10_c201612011122.avg_rade9h.tif
   Winsorizing: 1st percentile = 0.01, 99th percentile = 7.25
 

## Generate Winsorized Nighttime Light Maps

In [142]:
maps_created = 0

for year in available_years
    if haskey(VIIRS_FILES, year)
        viirs_file = joinpath(BASE_DIR, VIIRS_FILES[year])
        
        if isfile(viirs_file)
            try
                println("Creating map for $year...")
                
                # Load and process raster (same as Assignment2_Europa)
                raster_full = Raster(viirs_file; lazy=true)
                laos_raster = raster_full[laos_bounds...]
                
                # Winsorize data
                data = collect(laos_raster)
                non_zero_data = data[data .> 0]
                
                if length(non_zero_data) > 0
                    p1_val = percentile(non_zero_data, 1)
                    p99_val = percentile(non_zero_data, 99)
                    
                    # Apply winsorizing
                    data[data .> p99_val] .= p99_val
                    data[(data .> 0) .& (data .< p1_val)] .= p1_val
                    
                    # Rebuild raster
                    laos_raster_wins = Rasters.rebuild(laos_raster, data)
                    
                    # Mask to Laos boundaries
                    laos_country_geom = laos_country_shp.geometry[1]
                    laos_masked = mask(laos_raster_wins, with=laos_country_geom)
                    laos_masked_clean = replace_missing(laos_masked, 0)
                    
                    # Create the map
                    p = plot(laos_masked_clean, 
                            title="Laos Nighttime Lights - $year (Winsorized)\\nwith Administrative Boundaries",
                            color=:hot,
                            xlabel="Longitude",
                            ylabel="Latitude",
                            size=(1000, 800),
                            dpi=200,
                            margin=10Plots.mm,
                            titlefontsize=14,
                            guidefontsize=12,
                            left_margin=15Plots.mm,
                            bottom_margin=12Plots.mm)
                    
                    # Add province boundaries (white lines)
                    plot!(p, laos_provinces_shp.geometry, 
                          color=:transparent, 
                          linewidth=1, 
                          linecolor=:white,
                          linealpha=0.8)
                    
                    # Add country boundary (yellow lines)
                    plot!(p, laos_country_shp.geometry, 
                          color=:transparent, 
                          linewidth=2, 
                          linecolor=:yellow,
                          linealpha=0.9)
                    
                    # Save map
                    map_path = joinpath(OUTPUT_DIR, "maps", "laos_winsorized_lights_$year.png")
                    savefig(p, map_path)
                    
                    println("   Created winsorized map: laos_winsorized_lights_$year.png")
                    maps_created += 1
                else
                    println("   No valid light data for $year")
                end
                
            catch e
                println("   Error creating map for $year: $e")
            end
        else
            println("   VIIRS file not found for $year")
        end
    end
end

Creating map for 2012...
   Created winsorized map: laos_winsorized_lights_2012.png
Creating map for 2013...
   Created winsorized map: laos_winsorized_lights_2013.png
Creating map for 2014...
   Created winsorized map: laos_winsorized_lights_2014.png
Creating map for 2015...
   Created winsorized map: laos_winsorized_lights_2015.png
Creating map for 2016...
   Created winsorized map: laos_winsorized_lights_2016.png
Creating map for 2017...
   Created winsorized map: laos_winsorized_lights_2017.png
Creating map for 2018...
   Created winsorized map: laos_winsorized_lights_2018.png
Creating map for 2019...
   Created winsorized map: laos_winsorized_lights_2019.png
Creating map for 2020...
   Created winsorized map: laos_winsorized_lights_2020.png
Creating map for 2021...
   Created winsorized map: laos_winsorized_lights_2021.png
Creating map for 2022...
   Created winsorized map: laos_winsorized_lights_2022.png


## Henderson et al. (2012) idea's GDP Allocation

In [111]:
gdp_allocation = DataFrame()

for year in available_years
    # Get GDP data for the year
    gdp_year = filter(row -> row.year == year, gdp_data)
    if nrow(gdp_year) == 0
        continue
    end
    
    national_gdp = gdp_year[1, :gdp_current_usd]
    year_light_data = filter(row -> row.year == year, all_light_data)
    
    if nrow(year_light_data) == 0
        continue
    end
    
    # Calculate national totals
    national_light = sum(year_light_data.total_light)
    national_population = sum(year_light_data.population)
    national_area = sum(year_light_data.area_km2)
    
    println("   $year: GDP=\$$(round(national_gdp/1e9, digits=2))B, Light=$(round(national_light)), Pop=$(round(national_population/1e6, digits=2))M")
    
    if national_light > 0 && national_population > 0 && national_area > 0
        for row in eachrow(year_light_data)
            # Calculate shares (Henderson methodology)
            light_share = row.total_light / national_light
            pop_share = row.population / national_population
            area_share = row.area_km2 / national_area
            
            # Henderson combined weight: 60% light + 30% population + 10% area
            combined_weight = (WEIGHTS[:light] * light_share + 
                             WEIGHTS[:population] * pop_share + 
                             WEIGHTS[:area] * area_share)
            
            push!(gdp_allocation, merge(row, (
                national_gdp = national_gdp,
                light_share = light_share,
                population_share = pop_share,
                area_share = area_share,
                combined_weight = combined_weight
            )))
        end
    end
end

# Normalize weights and calculate final GDP allocation
gdp_allocation.final_weight = zeros(nrow(gdp_allocation))
gdp_allocation.allocated_gdp = zeros(nrow(gdp_allocation))
gdp_allocation.gdp_per_capita = zeros(nrow(gdp_allocation))
gdp_allocation.gdp_per_km2 = zeros(nrow(gdp_allocation))

for year in available_years
    year_mask = gdp_allocation.year .== year
    if sum(year_mask) > 0
        total_weight = sum(gdp_allocation[year_mask, :combined_weight])
        
        if total_weight > 0
            gdp_allocation[year_mask, :final_weight] = 
                gdp_allocation[year_mask, :combined_weight] ./ total_weight
            gdp_allocation[year_mask, :allocated_gdp] = 
                gdp_allocation[year_mask, :final_weight] .* gdp_allocation[year_mask, :national_gdp]
            gdp_allocation[year_mask, :gdp_per_capita] = 
                gdp_allocation[year_mask, :allocated_gdp] ./ gdp_allocation[year_mask, :population]
            gdp_allocation[year_mask, :gdp_per_km2] = 
                gdp_allocation[year_mask, :allocated_gdp] ./ gdp_allocation[year_mask, :area_km2]
        end
    end
end

println("\nHenderson GDP allocation completed: $(nrow(gdp_allocation)) records")
println("Weights used - Light: $(WEIGHTS[:light]*100)%, Population: $(WEIGHTS[:population]*100)%, Area: $(WEIGHTS[:area]*100)%")

# Validation check
for year in available_years
    year_data = filter(row -> row.year == year, gdp_allocation)
    if nrow(year_data) > 0
        allocated_sum = sum(year_data.allocated_gdp)
        national_gdp = year_data[1, :national_gdp]
        conservation_error = abs(allocated_sum - national_gdp) / national_gdp * 100
        
        if conservation_error < 0.01
            println("   $year: GDP conservation check passed (error: $(round(conservation_error, digits=4))%)")
        else
            println("   $year: GDP conservation error: $(round(conservation_error, digits=2))%")
        end
    end
end

   2012: GDP=$10.19B, Light=123546.0, Pop=5.88M
   2013: GDP=$11.98B, Light=128534.0, Pop=5.96M
   2014: GDP=$13.28B, Light=131984.0, Pop=6.05M
   2015: GDP=$14.43B, Light=130900.0, Pop=6.14M
   2016: GDP=$15.91B, Light=130440.0, Pop=6.24M
   2017: GDP=$17.07B, Light=142710.0, Pop=6.33M
   2018: GDP=$18.14B, Light=137810.0, Pop=6.43M
   2019: GDP=$18.74B, Light=138192.0, Pop=6.52M
   2020: GDP=$18.98B, Light=144073.0, Pop=6.62M
   2021: GDP=$18.83B, Light=142288.0, Pop=6.72M
   2022: GDP=$15.47B, Light=148941.0, Pop=6.82M

Henderson GDP allocation completed: 198 records
Weights used - Light: 60.0%, Population: 30.0%, Area: 10.0%
   2012: GDP conservation check passed (error: 0.0%)
   2013: GDP conservation check passed (error: 0.0%)
   2014: GDP conservation check passed (error: 0.0%)
   2015: GDP conservation check passed (error: 0.0%)
   2016: GDP conservation check passed (error: 0.0%)
   2017: GDP conservation check passed (error: 0.0%)
   2018: GDP conservation check passed (error

## Growth Analysis and Econometric Modeling

In [112]:
# Initialize variables at global scope
light_elasticity = NaN
r_squared = NaN
intercept = NaN
valid_data = DataFrame()

# Growth analysis
growth_analysis = DataFrame()

for province in unique(gdp_allocation.province_name)
    prov_data = filter(row -> row.province_name == province, gdp_allocation)
    sort!(prov_data, :year)
    
    if nrow(prov_data) >= 2
        start_row = prov_data[1, :]
        end_row = prov_data[end, :]
        years_span = end_row.year - start_row.year
        
        # Calculate CAGR
        gdp_cagr = if start_row.allocated_gdp > 0 && end_row.allocated_gdp > 0
            ((end_row.allocated_gdp / start_row.allocated_gdp)^(1/years_span) - 1) * 100
        else
            NaN
        end
        
        light_cagr = if start_row.total_light > 0 && end_row.total_light > 0
            ((end_row.total_light / start_row.total_light)^(1/years_span) - 1) * 100
        else
            NaN
        end
        
        push!(growth_analysis, (
            province_name = province,
            start_year = start_row.year,
            end_year = end_row.year,
            start_gdp_millions = start_row.allocated_gdp / 1e6,
            end_gdp_millions = end_row.allocated_gdp / 1e6,
            start_light = start_row.total_light,
            end_light = end_row.total_light,
            gdp_cagr = gdp_cagr,
            light_cagr = light_cagr,
            avg_gdp_per_capita = mean(prov_data.gdp_per_capita),
            correlation_gdp_light = cor(prov_data.allocated_gdp, prov_data.total_light)
        ))
    end
end

sort!(growth_analysis, :gdp_cagr, rev=true)
println("Growth analysis completed: $(nrow(growth_analysis)) provinces")

# Calculate Gini coefficients
gini_results = DataFrame()

for year in available_years
    year_data = filter(row -> row.year == year, gdp_allocation)
    if nrow(year_data) > 0
        values = sort(year_data.allocated_gdp)
        n = length(values)
        index = 1:n
        gini = (2 * sum(index .* values)) / (n * sum(values)) - (n + 1) / n
        push!(gini_results, (year = year, gini_coefficient = gini))
    end
end

println("Gini coefficients calculated: $(nrow(gini_results)) years")

# Econometric analysis: log(GDP) ~ log(Light)
global valid_data = filter(row -> row.allocated_gdp > 0 && row.total_light > 0, gdp_allocation)

if nrow(valid_data) > 0
    log_gdp = log.(valid_data.allocated_gdp)
    log_light = log.(valid_data.total_light)
    
    # Simple regression: β = (X_matrix'X_matrix)^(-1)X_matrix'y
    # Rename X to X_matrix to avoid conflict with Rasters.X
    X_matrix = hcat(ones(length(log_light)), log_light)
    β = (X_matrix' * X_matrix) \ (X_matrix' * log_gdp)
    
    # Calculate R-squared
    y_pred = X_matrix * β
    ss_res = sum((log_gdp .- y_pred).^2)
    ss_tot = sum((log_gdp .- mean(log_gdp)).^2)
    global r_squared = 1 - ss_res / ss_tot
    
    global light_elasticity = β[2]
    global intercept = β[1]
    
    println("Econometric analysis completed:")
    println("   Light elasticity: $(round(light_elasticity, digits=4))")
    println("   R²: $(round(r_squared, digits=4))")
    println("   Sample size: $(nrow(valid_data)) observations")
    println("   Intercept: $(round(intercept, digits=4))")
else
    global light_elasticity = NaN
    global r_squared = NaN
    println("No valid data for econometric analysis")
end

Growth analysis completed: 18 provinces
Gini coefficients calculated: 11 years
Econometric analysis completed:
   Light elasticity: 0.8138
   R²: 0.5594
   Sample size: 198 observations
   Intercept: 13.2895


## Data Visualizations - Time Series Analysis 2012-2022

In [113]:
# 1. GDP trends over time (all provinces)
p1 = plot(title="Provincial GDP Trends (2012-2022)\nHenderson et al. (2012) Methodology",
          xlabel="Year", ylabel="GDP (Million USD)",
          size=(1200, 800), dpi=200,
          margin=10Plots.mm, left_margin=15Plots.mm, bottom_margin=12Plots.mm)

# Get top 5 provinces for highlighting
province_totals = combine(groupby(gdp_allocation, :province_name), :allocated_gdp => sum => :total_gdp)
sort!(province_totals, :total_gdp, rev=true)
top_provinces = province_totals[1:5, :province_name]

colors = [:blue, :red, :green, :orange, :purple]

# Plot all provinces (gray lines for non-top provinces)
for province in unique(gdp_allocation.province_name)
    prov_data = filter(row -> row.province_name == province, gdp_allocation)
    sort!(prov_data, :year)
    
    if province in top_provinces
        idx = findfirst(==(province), top_provinces)
        plot!(p1, prov_data.year, prov_data.allocated_gdp ./ 1e6,
              label=province, linewidth=3, color=colors[idx],
              marker=:circle, markersize=5)
    else
        plot!(p1, prov_data.year, prov_data.allocated_gdp ./ 1e6,
              label="", linewidth=1.5, alpha=0.4, color=:gray)
    end
end

savefig(p1, joinpath(OUTPUT_DIR, "plots", "time_series_gdp_trends.png"))

# 2. Light intensity trends over time
p2 = plot(title="Provincial Light Intensity Trends (2012-2022)",
          xlabel="Year", ylabel="Total Light Intensity",
          size=(1200, 800), dpi=200,
          margin=10Plots.mm, left_margin=15Plots.mm, bottom_margin=12Plots.mm)

for province in unique(gdp_allocation.province_name)
    prov_data = filter(row -> row.province_name == province, gdp_allocation)
    sort!(prov_data, :year)
    
    if province in top_provinces
        idx = findfirst(==(province), top_provinces)
        plot!(p2, prov_data.year, prov_data.total_light,
              label=province, linewidth=3, color=colors[idx],
              marker=:circle, markersize=5)
    else
        plot!(p2, prov_data.year, prov_data.total_light,
              label="", linewidth=1.5, alpha=0.4, color=:gray)
    end
end

savefig(p2, joinpath(OUTPUT_DIR, "plots", "time_series_light_trends.png"))

# 3. Gini coefficient evolution
p3 = plot(gini_results.year, gini_results.gini_coefficient,
          title="Economic Inequality Evolution\n(Gini Coefficient of GDP Distribution)",
          xlabel="Year", ylabel="Gini Coefficient",
          linewidth=3, marker=:circle, markersize=7,
          color=:darkred, legend=false,
          size=(1000, 600), dpi=200,
          margin=10Plots.mm, left_margin=15Plots.mm, bottom_margin=12Plots.mm)

savefig(p3, joinpath(OUTPUT_DIR, "plots", "time_series_gini_evolution.png"))

# 4. GDP per capita evolution by province
p4 = plot(title="GDP per Capita Trends (2012-2022)",
          xlabel="Year", ylabel="GDP per Capita (USD)",
          size=(1200, 800), dpi=200,
          margin=10Plots.mm, left_margin=15Plots.mm, bottom_margin=12Plots.mm)

for province in top_provinces
    prov_data = filter(row -> row.province_name == province, gdp_allocation)
    sort!(prov_data, :year)
    idx = findfirst(==(province), top_provinces)
    
    plot!(p4, prov_data.year, prov_data.gdp_per_capita,
          label=province, linewidth=3, color=colors[idx],
          marker=:circle, markersize=5)
end

savefig(p4, joinpath(OUTPUT_DIR, "plots", "time_series_gdp_per_capita.png"))

"/Users/ouyangwendi/Desktop/A-Europa Final Project Framework/OUTPUT/plots/time_series_gdp_per_capita.png"

## Data Visualizations - Provincial Comparisons

In [114]:
# 1. Average GDP by province (with different years shown in colors)
# Create a grouped bar chart showing GDP for different years
selected_years = [2012, 2016, 2020, 2022]
province_names = unique(gdp_allocation.province_name)
year_colors = [:lightblue, :steelblue, :darkblue, :navy]

p5 = plot(title="GDP by Province Across Years\nHenderson et al. (2012) Methodology",
          xlabel="Province", ylabel="GDP (Million USD)",
          size=(1400, 800), dpi=200,
          margin=10Plots.mm, left_margin=20Plots.mm, bottom_margin=20Plots.mm)

# Calculate positions for grouped bars
x_positions = 1:length(province_names)
bar_width = 0.2

for (i, year) in enumerate(selected_years)
    year_data = filter(row -> row.year == year, gdp_allocation)
    gdp_values = Float64[]
    
    for province in province_names
        prov_gdp = filter(row -> row.province_name == province, year_data)
        if nrow(prov_gdp) > 0
            push!(gdp_values, prov_gdp[1, :allocated_gdp] / 1e6)
        else
            push!(gdp_values, 0.0)
        end
    end
    
    offset = (i - 2.5) * bar_width
    bar!(p5, x_positions .+ offset, gdp_values,
         bar_width=bar_width, label="$year", color=year_colors[i], alpha=0.8)
end

plot!(p5, xticks=(x_positions, province_names), xrotation=45)
savefig(p5, joinpath(OUTPUT_DIR, "plots", "provincial_gdp_comparison_years.png"))

# 2. Light intensity by province (with different years)
p6 = plot(title="Light Intensity by Province Across Years",
          xlabel="Province", ylabel="Total Light Intensity",
          size=(1400, 800), dpi=200,
          margin=10Plots.mm, left_margin=20Plots.mm, bottom_margin=20Plots.mm)

for (i, year) in enumerate(selected_years)
    year_data = filter(row -> row.year == year, gdp_allocation)
    light_values = Float64[]
    
    for province in province_names
        prov_light = filter(row -> row.province_name == province, year_data)
        if nrow(prov_light) > 0
            push!(light_values, prov_light[1, :total_light])
        else
            push!(light_values, 0.0)
        end
    end
    
    offset = (i - 2.5) * bar_width
    bar!(p6, x_positions .+ offset, light_values,
         bar_width=bar_width, label="$year", color=year_colors[i], alpha=0.8)
end

plot!(p6, xticks=(x_positions, province_names), xrotation=45)
savefig(p6, joinpath(OUTPUT_DIR, "plots", "provincial_light_comparison_years.png"))

# 3. Growth rates comparison
valid_growth = filter(row -> !isnan(row.gdp_cagr), growth_analysis)
sort!(valid_growth, :gdp_cagr, rev=true)

p7 = bar(valid_growth.province_name, valid_growth.gdp_cagr,
         title="Provincial GDP Growth Rates (2012-2022)\nCompound Annual Growth Rate (%)",
         xlabel="Province", ylabel="GDP CAGR (%)",
         xrotation=45, legend=false,
         size=(1400, 800), dpi=200,
         color=:darkgreen, alpha=0.8,
         margin=10Plots.mm, left_margin=20Plots.mm, bottom_margin=20Plots.mm)

savefig(p7, joinpath(OUTPUT_DIR, "plots", "provincial_growth_rates.png"))

# 4. GDP per capita comparison (latest year)
latest_year_data = filter(row -> row.year == 2022, gdp_allocation)
sort!(latest_year_data, :gdp_per_capita, rev=true)

p8 = bar(latest_year_data.province_name, latest_year_data.gdp_per_capita,
         title="GDP per Capita by Province (2022)\nHenderson et al. (2012) Methodology",
         xlabel="Province", ylabel="GDP per Capita (USD)",
         xrotation=45, legend=false,
         size=(1400, 800), dpi=200,
         color=:steelblue, alpha=0.8,
         margin=10Plots.mm, left_margin=20Plots.mm, bottom_margin=20Plots.mm)

savefig(p8, joinpath(OUTPUT_DIR, "plots", "provincial_gdp_per_capita_2022.png"))

"/Users/ouyangwendi/Desktop/A-Europa Final Project Framework/OUTPUT/plots/provincial_gdp_per_capita_2022.png"

In [115]:
# GDP vs Light correlation scatter plot
p9 = scatter(gdp_allocation.total_light, gdp_allocation.allocated_gdp ./ 1e6,
            xlabel="Total Light Intensity", ylabel="GDP (Million USD)",
            title="GDP vs Nighttime Light Intensity (2012-2022)\nHenderson et al. (2012) Methodology",
            markersize=4, markeralpha=0.7, color=:steelblue,
            size=(1000, 700), dpi=200, legend=false,
            margin=10Plots.mm, left_margin=15Plots.mm, bottom_margin=12Plots.mm)

# Add trend line
x_vals = gdp_allocation.total_light
y_vals = gdp_allocation.allocated_gdp ./ 1e6
m = sum((x_vals .- mean(x_vals)) .* (y_vals .- mean(y_vals))) / sum((x_vals .- mean(x_vals)).^2)
b = mean(y_vals) - m * mean(x_vals)

x_range = range(minimum(x_vals), maximum(x_vals), length=100)
plot!(p9, x_range, m .* x_range .+ b, color=:red, linewidth=3)

# Add R² annotation
if !isnan(r_squared)
    annotate!(p9, maximum(x_vals) * 0.7, maximum(y_vals) * 0.9, 
             text("R² = $(round(r_squared, digits=3))\nElasticity = $(round(light_elasticity, digits=3))", 12))
end

savefig(p9, joinpath(OUTPUT_DIR, "plots", "gdp_light_correlation.png"))

# Spatial visualization (2022 data)
year_2022_data = filter(row -> row.year == 2022, gdp_allocation)

p10 = scatter(year_2022_data.longitude, year_2022_data.latitude,
             markersize = sqrt.(year_2022_data.total_light) ./ 100,
             marker_z = year_2022_data.allocated_gdp ./ 1e6,
             markershape = :circle, markerstrokewidth = 1,
             markerstrokecolor = :black, color = :hot,
             title = "Laos Provincial Economic Activity - 2022\n(Size: Light Intensity, Color: GDP)",
             xlabel = "Longitude", ylabel = "Latitude",
             size=(1000, 800), dpi = 200,
             margin=10Plots.mm, left_margin=15Plots.mm, bottom_margin=12Plots.mm)

# Add province labels for major provinces
for row in eachrow(year_2022_data)
    if row.total_light > quantile(year_2022_data.total_light, 0.6)
        annotate!(p10, row.longitude, row.latitude + 0.15, 
                 text(row.province_name, 8, :black, :center))
    end
end

savefig(p10, joinpath(OUTPUT_DIR, "plots", "spatial_economic_activity_2022.png"))

# Economic development trajectory (bubble chart)
# Show GDP vs GDP per capita with bubble size as light intensity
p11 = plot(title="Economic Development Trajectory (2022)\nBubble Size: Light Intensity",
          xlabel="GDP per Capita (USD)", ylabel="Total GDP (Million USD)",
          size=(1000, 800), dpi=200,
          margin=10Plots.mm, left_margin=15Plots.mm, bottom_margin=12Plots.mm)

scatter!(p11, year_2022_data.gdp_per_capita, year_2022_data.allocated_gdp ./ 1e6,
        markersize = sqrt.(year_2022_data.total_light) ./ 50,
        markeralpha = 0.7, color = :steelblue,
        markerstrokewidth = 1, markerstrokecolor = :black,
        legend = false)

# Add labels for top provinces
top_gdp_provinces = sort(year_2022_data, :allocated_gdp, rev=true)[1:5, :]
for row in eachrow(top_gdp_provinces)
    annotate!(p11, row.gdp_per_capita + 50, row.allocated_gdp / 1e6 + 50,
             text(row.province_name, 8, :black, :left))
end

savefig(p11, joinpath(OUTPUT_DIR, "plots", "economic_development_trajectory.png"))


"/Users/ouyangwendi/Desktop/A-Europa Final Project Framework/OUTPUT/plots/economic_development_trajectory.png"

## Export Data Files

In [116]:
# Create BOKEOMAP directory
bokeo_map_dir = joinpath(OUTPUT_DIR, "BOKEOMAP")
if !isdir(bokeo_map_dir)
    mkpath(bokeo_map_dir)
end

# 1. Extract Bokeo data for regression analysis
bokeo_data = filter(row -> row.province_name == "Bokeo", gdp_allocation)
sort!(bokeo_data, :year)

# 1.1 Bokeo GDP vs Nightlight Regression
if nrow(bokeo_data) > 0
    bokeo_gdp = bokeo_data.allocated_gdp
    bokeo_light = bokeo_data.total_light
    
    # Filter valid data points
    valid_mask = (bokeo_gdp .> 0) .& (bokeo_light .> 0)
    valid_gdp = bokeo_gdp[valid_mask]
    valid_light = bokeo_light[valid_mask]
    
    if length(valid_gdp) >= 2
        # Linear regression
        X_bokeo = hcat(ones(length(valid_light)), valid_light)
        β_bokeo = (X_bokeo' * X_bokeo) \ (X_bokeo' * valid_gdp)
        
        # Calculate R-squared
        y_pred_bokeo = X_bokeo * β_bokeo
        ss_res_bokeo = sum((valid_gdp .- y_pred_bokeo).^2)
        ss_tot_bokeo = sum((valid_gdp .- mean(valid_gdp)).^2)
        r_squared_bokeo = 1 - ss_res_bokeo / ss_tot_bokeo
        
        println("Bokeo Regression Results:")
        println("   Light coefficient: $(round(β_bokeo[2], digits=6))")
        println("   Intercept: $(round(β_bokeo[1], digits=2))")
        println("   R²: $(round(r_squared_bokeo, digits=4))")
        
        # Log-linear regression for elasticity
        if all(valid_gdp .> 0) && all(valid_light .> 0)
            log_gdp_bokeo = log.(valid_gdp)
            log_light_bokeo = log.(valid_light)
            
            X_log_bokeo = hcat(ones(length(log_light_bokeo)), log_light_bokeo)
            β_log_bokeo = (X_log_bokeo' * X_log_bokeo) \ (X_log_bokeo' * log_gdp_bokeo)
            
            y_pred_log_bokeo = X_log_bokeo * β_log_bokeo
            ss_res_log_bokeo = sum((log_gdp_bokeo .- y_pred_log_bokeo).^2)
            ss_tot_log_bokeo = sum((log_gdp_bokeo .- mean(log_gdp_bokeo)).^2)
            r_squared_log_bokeo = 1 - ss_res_log_bokeo / ss_tot_log_bokeo
            
            bokeo_elasticity = β_log_bokeo[2]
            println("   Light elasticity: $(round(bokeo_elasticity, digits=4))")
            println("   Log-regression R²: $(round(r_squared_log_bokeo, digits=4))")
        end
    end
end

# 2. Extract Bokeo province boundary for mapping
bokeo_province_shp = filter(row -> occursin("Bokeo", string(row.ADM1_EN)), laos_provinces_shp)
if nrow(bokeo_province_shp) == 0
    # Try alternative name matching
    bokeo_province_shp = filter(row -> occursin("BOKEO", uppercase(string(row.ADM1_EN))), laos_provinces_shp)
end

println("Bokeo province boundary found: $(nrow(bokeo_province_shp)) record(s)")

# Define Bokeo bounding box (from province coordinates)
bokeo_coords = filter(p -> p[1] == "Bokeo", provinces)[1]
bokeo_lat, bokeo_lon = bokeo_coords[2], bokeo_coords[3]

# Create expanded bounding box for Bokeo region
bokeo_buffer = 0.5  # degrees
bokeo_lon_range = (bokeo_lon - bokeo_buffer)..(bokeo_lon + bokeo_buffer)
bokeo_lat_range = (bokeo_lat - bokeo_buffer)..(bokeo_lat + bokeo_buffer)
bokeo_bounds = (X(bokeo_lon_range), Y(bokeo_lat_range))

println("Bokeo bounding box: $(round(bokeo_lon - bokeo_buffer, digits=2))-$(round(bokeo_lon + bokeo_buffer, digits=2))°E, $(round(bokeo_lat - bokeo_buffer, digits=2))-$(round(bokeo_lat + bokeo_buffer, digits=2))°N")

# 3. Create Bokeo regional maps for each year
bokeo_maps_created = 0

for year in available_years
    if haskey(VIIRS_FILES, year)
        viirs_file = joinpath(BASE_DIR, VIIRS_FILES[year])
        
        if isfile(viirs_file)
            try
                println("Creating Bokeo map for $year...")
                
                # Load and process raster
                raster_full = Raster(viirs_file; lazy=true)
                laos_raster = raster_full[laos_bounds...]
                
                # Winsorize data
                data = collect(laos_raster)
                non_zero_data = data[data .> 0]
                
                if length(non_zero_data) > 0
                    p1_val = percentile(non_zero_data, 1)
                    p99_val = percentile(non_zero_data, 99)
                    
                    # Apply winsorizing
                    data[data .> p99_val] .= p99_val
                    data[(data .> 0) .& (data .< p1_val)] .= p1_val
                    
                    # Rebuild raster
                    laos_raster_wins = Rasters.rebuild(laos_raster, data)
                    
                    # Extract Bokeo region
                    try
                        bokeo_raster = laos_raster_wins[bokeo_bounds...]
                        
                        # Create the Bokeo map
                        p_bokeo = plot(bokeo_raster, 
                                title="Bokeo Province Nighttime Lights - $year",
                                color=:hot,
                                xlabel="Longitude",
                                ylabel="Latitude",
                                size=(800, 800),
                                dpi=200,
                                margin=10Plots.mm,
                                titlefontsize=14,
                                guidefontsize=12)
                        
                        # Add Bokeo province boundary if available
                        if nrow(bokeo_province_shp) > 0
                            plot!(p_bokeo, bokeo_province_shp.geometry, 
                                  color=:transparent, 
                                  linewidth=2, 
                                  linecolor=:yellow,
                                  linealpha=0.9)
                        end
                        
                        # Add Bokeo center point
                        scatter!(p_bokeo, [bokeo_lon], [bokeo_lat],
                                marker=:star, markersize=8, markercolor=:white,
                                markerstrokewidth=2, markerstrokecolor=:black,
                                label="Bokeo Center")
                        
                        # Save Bokeo map
                        bokeo_map_path = joinpath(bokeo_map_dir, "bokeo_nightlights_$year.png")
                        savefig(p_bokeo, bokeo_map_path)
                        
                        println("   Created Bokeo map: bokeo_nightlights_$year.png")
                        bokeo_maps_created += 1
                    catch e
                        println("   Error extracting Bokeo region for $year: $e")
                    end
                else
                    println("   No valid light data for $year")
                end
            catch e
                println("   Error processing Bokeo map for $year: $e")
            end
        end
    end
end

# 4. Create dual-axis plot: Bokeo GDP vs Nightlight over time
if nrow(bokeo_data) > 0
    # Normalize data for dual-axis visualization
    bokeo_years = bokeo_data.year
    bokeo_gdp_millions = bokeo_data.allocated_gdp ./ 1e6
    bokeo_lights = bokeo_data.total_light
    
    # Create dual-axis plot
    p_dual = plot(bokeo_years, bokeo_gdp_millions,
                 ylabel="GDP (Million USD)",
                 xlabel="Year",
                 title="Bokeo Province: GDP vs Nighttime Lights (2012-2022)",
                 linewidth=3, marker=:circle, markersize=6,
                 color=:blue, label="GDP (Million USD)",
                 size=(1200, 800), dpi=200,
                 margin=15Plots.mm)
    
    # Add second y-axis for lights
    p_dual_twin = twinx(p_dual)
    plot!(p_dual_twin, bokeo_years, bokeo_lights,
          ylabel="Total Light Intensity",
          linewidth=3, marker=:square, markersize=6,
          color=:red, label="Light Intensity",
          legend=:topright)
    
    # Save dual-axis plot
    dual_axis_path = joinpath(OUTPUT_DIR, "plots", "bokeo_gdp_light_dual_axis.png")
    savefig(p_dual, dual_axis_path)
    
    println("Created dual-axis plot: bokeo_gdp_light_dual_axis.png")
end

# 5. Calculate and visualize Growth Gap Analysis
if nrow(bokeo_data) >= 2
    # Calculate year-over-year growth rates
    bokeo_growth_analysis = DataFrame()
    
    for i in 2:nrow(bokeo_data)
        prev_year_data = bokeo_data[i-1, :]
        curr_year_data = bokeo_data[i, :]
        
        # Calculate growth rates
        gdp_growth_rate = if prev_year_data.allocated_gdp > 0
            ((curr_year_data.allocated_gdp / prev_year_data.allocated_gdp) - 1) * 100
        else
            NaN
        end
        
        light_growth_rate = if prev_year_data.total_light > 0
            ((curr_year_data.total_light / prev_year_data.total_light) - 1) * 100
        else
            NaN
        end
        
        growth_gap = light_growth_rate - gdp_growth_rate
        
        push!(bokeo_growth_analysis, (
            year = curr_year_data.year,
            gdp_growth = gdp_growth_rate,
            light_growth = light_growth_rate,
            growth_gap = growth_gap
        ))
    end
    
    println("Growth gap analysis completed: $(nrow(bokeo_growth_analysis)) periods")
    
    # Create growth gap visualization
    if nrow(bokeo_growth_analysis) > 0
        p_gap = plot(bokeo_growth_analysis.year, bokeo_growth_analysis.gdp_growth,
                    label="GDP Growth Rate (%)", linewidth=3, marker=:circle,
                    color=:blue, markersize=6,
                    title="Bokeo Province: Growth Rate Analysis",
                    xlabel="Year", ylabel="Growth Rate (%)",
                    size=(1200, 800), dpi=200,
                    margin=15Plots.mm)
        
        plot!(p_gap, bokeo_growth_analysis.year, bokeo_growth_analysis.light_growth,
              label="Light Growth Rate (%)", linewidth=3, marker=:square,
              color=:red, markersize=6)
        
        plot!(p_gap, bokeo_growth_analysis.year, bokeo_growth_analysis.growth_gap,
              label="Growth Gap (Light - GDP)", linewidth=3, marker=:diamond,
              color=:green, markersize=6, linestyle=:dash)
        
        # Add zero line
        hline!(p_gap, [0], color=:black, linestyle=:dot, alpha=0.5, label="")
        
        # Save growth gap plot
        growth_gap_path = joinpath(OUTPUT_DIR, "plots", "bokeo_growth_gap_analysis.png")
        savefig(p_gap, growth_gap_path)
        
        println("Created growth gap analysis: bokeo_growth_gap_analysis.png")
        
        # Calculate summary statistics
        avg_gdp_growth = mean(filter(!isnan, bokeo_growth_analysis.gdp_growth))
        avg_light_growth = mean(filter(!isnan, bokeo_growth_analysis.light_growth))
        avg_growth_gap = mean(filter(!isnan, bokeo_growth_analysis.growth_gap))
        
        println("\nBokeo Growth Summary:")
        println("   Average GDP growth: $(round(avg_gdp_growth, digits=2))%")
        println("   Average light growth: $(round(avg_light_growth, digits=2))%")
        println("   Average growth gap: $(round(avg_growth_gap, digits=2))%")
    end
    
    # Export Bokeo analysis data
    CSV.write(joinpath(OUTPUT_DIR, "data", "bokeo_growth_gap_analysis.csv"), bokeo_growth_analysis)
end

println("\nBokeo Province Analysis Summary:")
println("   Bokeo maps created: $bokeo_maps_created")
println("   Maps saved to: $bokeo_map_dir")
println("   Dual-axis visualization: bokeo_gdp_light_dual_axis.png")
println("   Growth gap analysis: bokeo_growth_gap_analysis.png")
println("   Data exported: bokeo_growth_gap_analysis.csv")

Bokeo Regression Results:
   Light coefficient: 208373.180594
   Intercept: -3.5291222941e8
   R²: 0.8555
   Light elasticity: 1.908
   Log-regression R²: 0.8434
Bokeo province boundary found: 1 record(s)
Bokeo bounding box: 99.92-100.92°E, 19.73-20.73°N
Creating Bokeo map for 2012...
   Created Bokeo map: bokeo_nightlights_2012.png
Creating Bokeo map for 2013...
   Created Bokeo map: bokeo_nightlights_2013.png
Creating Bokeo map for 2014...
   Created Bokeo map: bokeo_nightlights_2014.png
Creating Bokeo map for 2015...
   Created Bokeo map: bokeo_nightlights_2015.png
Creating Bokeo map for 2016...
   Created Bokeo map: bokeo_nightlights_2016.png
Creating Bokeo map for 2017...
   Created Bokeo map: bokeo_nightlights_2017.png
Creating Bokeo map for 2018...
   Created Bokeo map: bokeo_nightlights_2018.png
Creating Bokeo map for 2019...
   Created Bokeo map: bokeo_nightlights_2019.png
Creating Bokeo map for 2020...
   Created Bokeo map: bokeo_nightlights_2020.png
Creating Bokeo map for 20

## Bokeo Province Specialized Analysis

In [ ]:
# Calculate year-over-year growth rates for all provinces
growth_rates_data = DataFrame()

for province in unique(gdp_allocation.province_name)
    province_data = filter(row -> row.province_name == province, gdp_allocation)
    sort!(province_data, :year)
    
    if nrow(province_data) >= 2
        for i in 2:nrow(province_data)
            prev_year = province_data[i-1, :]
            curr_year = province_data[i, :]
            
            # Calculate GDP growth rate
            gdp_growth = if prev_year.allocated_gdp > 0
                ((curr_year.allocated_gdp / prev_year.allocated_gdp) - 1) * 100
            else
                NaN
            end
            
            # Calculate night light intensity growth rate
            light_growth = if prev_year.total_light > 0
                ((curr_year.total_light / prev_year.total_light) - 1) * 100
            else
                NaN
            end
            
            # Calculate growth gap
            growth_gap = light_growth - gdp_growth
            
            push!(growth_rates_data, (
                province = province,
                year = curr_year.year,
                gdp_growth_rate = gdp_growth,
                light_growth_rate = light_growth,
                growth_gap = growth_gap
            ))
        end
    end
end

# Filter out invalid data
valid_growth_data = filter(row -> !isnan(row.gdp_growth_rate) && !isnan(row.light_growth_rate), growth_rates_data)

# 1. Create GDP growth rate vs Night Light growth rate visualization with gap changes
# Aggregate national averages by year
national_growth = combine(groupby(valid_growth_data, :year),
    :gdp_growth_rate => mean => :avg_gdp_growth,
    :light_growth_rate => mean => :avg_light_growth,
    :growth_gap => mean => :avg_growth_gap
)
sort!(national_growth, :year)

# Create the combined growth rate plot
p_growth_comparison = plot(national_growth.year, national_growth.avg_gdp_growth,
    label="GDP Growth Rate (%)",
    linewidth=3,
    marker=:circle,
    markersize=6,
    color=:blue,
    title="National Average: GDP vs Night Light Intensity Growth Rates",
    xlabel="Year",
    ylabel="Growth Rate (%)",
    size=(1200, 800),
    dpi=200,
    margin=15Plots.mm,
    legend=:topleft)

plot!(p_growth_comparison, national_growth.year, national_growth.avg_light_growth,
    label="Night Light Growth Rate (%)",
    linewidth=3,
    marker=:square,
    markersize=6,
    color=:red)

plot!(p_growth_comparison, national_growth.year, national_growth.avg_growth_gap,
    label="Growth Gap (Light - GDP)",
    linewidth=2,
    marker=:diamond,
    markersize=5,
    color=:green,
    linestyle=:dash)

# Add zero reference line
hline!(p_growth_comparison, [0], color=:black, linestyle=:dot, alpha=0.5, label="")

savefig(p_growth_comparison, joinpath(OUTPUT_DIR, "plots", "gdp_nightlight_growth_rates_comparison.png"))
println("Created: gdp_nightlight_growth_rates_comparison.png")

# 2. Create time series plot of GDP and Night Light Intensity changes
# Aggregate national totals by year
national_totals = combine(groupby(gdp_allocation, :year),
    :allocated_gdp => sum => :total_gdp,
    :total_light => sum => :total_light_intensity
)
sort!(national_totals, :year)

# Convert GDP to millions for better readability
national_totals.gdp_millions = national_totals.total_gdp ./ 1e6

# Create dual-axis plot
p_time_series = plot(national_totals.year, national_totals.gdp_millions,
    ylabel="GDP (Million USD)",
    xlabel="Year",
    title="National GDP and Night Light Intensity Over Time",
    linewidth=3,
    marker=:circle,
    markersize=6,
    color=:blue,
    label="GDP (Million USD)",
    size=(1200, 800),
    dpi=200,
    margin=15Plots.mm,
    legend=:topleft)

# Add second y-axis for light intensity
p_time_series_twin = twinx(p_time_series)
plot!(p_time_series_twin, national_totals.year, national_totals.total_light_intensity,
    ylabel="Total Light Intensity",
    linewidth=3,
    marker=:square,
    markersize=6,
    color=:red,
    label="Night Light Intensity",
    legend=:topright)

savefig(p_time_series, joinpath(OUTPUT_DIR, "plots", "gdp_nightlight_time_series.png"))
println("Created: gdp_nightlight_time_series.png")

# 3. Create 3D visualization for linear regression
# Prepare data for 3D plot - use growth rates
valid_3d_data = filter(row -> !isnan(row.gdp_growth_rate) && !isnan(row.light_growth_rate), growth_rates_data)

# Create 3D scatter plot
p_3d = scatter3d(valid_3d_data.year,
    valid_3d_data.gdp_growth_rate,
    valid_3d_data.light_growth_rate,
    xlabel="Year",
    ylabel="GDP Change Rate (%)",
    zlabel="Night Light Change Rate (%)",
    title="3D Visualization: Year vs GDP Change vs Night Light Change",
    markersize=3,
    markercolor=:viridis,
    markerstrokewidth=0,
    camera=(30, 30),
    size=(1000, 800),
    dpi=200,
    margin=10Plots.mm)

# Perform linear regression for the 3D relationship
X_3d = hcat(ones(length(valid_3d_data.year)), 
            valid_3d_data.year .- minimum(valid_3d_data.year),  # Center years for numerical stability
            valid_3d_data.gdp_growth_rate)
y_3d = valid_3d_data.light_growth_rate

# Calculate regression coefficients
β_3d = (X_3d' * X_3d) \ (X_3d' * y_3d)

# Generate regression plane points
year_range = minimum(valid_3d_data.year):maximum(valid_3d_data.year)
gdp_range = range(minimum(valid_3d_data.gdp_growth_rate), 
                  maximum(valid_3d_data.gdp_growth_rate), 
                  length=20)

# Create mesh for regression plane
year_mesh = repeat(year_range, 1, length(gdp_range))
gdp_mesh = repeat(gdp_range', length(year_range), 1)
light_pred = β_3d[1] .+ β_3d[2] .* (year_mesh .- minimum(valid_3d_data.year)) .+ β_3d[3] .* gdp_mesh

# Add regression plane
surface!(p_3d, year_range, gdp_range, light_pred',
    alpha=0.3,
    color=:red,
    colorbar=false)

savefig(p_3d, joinpath(OUTPUT_DIR, "plots", "gdp_nightlight_3d_regression.png"))
println("Created: gdp_nightlight_3d_regression.png")

# Calculate R-squared for 3D regression
y_pred_3d = X_3d * β_3d
ss_res_3d = sum((y_3d .- y_pred_3d).^2)
ss_tot_3d = sum((y_3d .- mean(y_3d)).^2)
r_squared_3d = 1 - ss_res_3d / ss_tot_3d

println("\n3D Linear Regression Results:")
println("   Intercept: $(round(β_3d[1], digits=3))")
println("   Year coefficient: $(round(β_3d[2], digits=3))")
println("   GDP growth coefficient: $(round(β_3d[3], digits=3))")
println("   R²: $(round(r_squared_3d, digits=4))")

# Export the growth rate data
CSV.write(joinpath(OUTPUT_DIR, "data", "gdp_nightlight_growth_rates.csv"), valid_growth_data)
CSV.write(joinpath(OUTPUT_DIR, "data", "national_growth_averages.csv"), national_growth)


Created: gdp_nightlight_growth_rates_comparison.png
Creating time series visualization...
Created: gdp_nightlight_time_series.png
Creating 3D linear regression visualization...
Created: gdp_nightlight_3d_regression.png

3D Linear Regression Results:
   Intercept: -18.708
   Year coefficient: 3.431
   GDP growth coefficient: 1.22
   R²: 0.6865


"/Users/ouyangwendi/Desktop/A-Europa Final Project Framework/OUTPUT/data/national_growth_averages.csv"

# Time Series Analysis comparison between Bokeo and other region in SEZ

In [143]:
# SEZ Core Area Analysis - Bokeo vs Non-Bokeo

# Define SEZ core area bounds
sez_core_bounds = (X(99.95..100.35), Y(20.05..20.45))

# Get Bokeo province
bokeo_province = filter(row -> row.ADM1_EN == "Bokeo", laos_provinces_shp)

# Initialize results
sez_core_data = DataFrame()

for year in available_years
    viirs_file = joinpath(BASE_DIR, VIIRS_FILES[year])
    
    if isfile(viirs_file)
        # Load and extract SEZ core area
        raster_full = Raster(viirs_file; lazy=true)
        sez_core_raster = raster_full[sez_core_bounds...]
        
        # Mask with Bokeo boundary
        if nrow(bokeo_province) > 0
            bokeo_masked = mask(sez_core_raster; with=bokeo_province.geometry[1])
            bokeo_data = collect(skipmissing(bokeo_masked))
            bokeo_light = length(bokeo_data) > 0 ? sum(bokeo_data[bokeo_data .> 0]) : 0.0
        else
            bokeo_light = 0.0
        end
        
        # Total core area light
        core_data = collect(sez_core_raster)
        total_light = sum(core_data[core_data .> 0])
        non_bokeo_light = total_light - bokeo_light
        
        push!(sez_core_data, (
            year = year,
            total_core_light = total_light,
            bokeo_core_light = bokeo_light,
            non_bokeo_core_light = non_bokeo_light,
            bokeo_percentage = (bokeo_light / total_light) * 100
        ))
    end
end

# Plot comparison
p = plot(sez_core_data.year, sez_core_data.bokeo_core_light,
    label="Bokeo (SEZ Core)",
    marker=:circle,
    linewidth=2,
    color=:red,
    title="SEZ Core Area: Bokeo vs Non-Bokeo",
    xlabel="Year",
    ylabel="Light Intensity",
    size=(800, 600))

plot!(p, sez_core_data.year, sez_core_data.non_bokeo_core_light,
    label="Non-Bokeo (SEZ Core)",
    marker=:square,
    linewidth=2,
    color=:blue)

savefig(p, joinpath(OUTPUT_DIR, "plots", "sez_core_comparison.png"))

# Create maps for 2012 and 2022
for year in [2012, 2022]
    viirs_file = joinpath(BASE_DIR, VIIRS_FILES[year])
    
    if isfile(viirs_file)
        # Load and process
        raster_full = Raster(viirs_file; lazy=true)
        core_raster = raster_full[sez_core_bounds...]
        
        # Winsorize
        data = collect(core_raster)
        non_zero = data[data .> 0]
        if length(non_zero) > 0
            p1 = percentile(non_zero, 1)
            p99 = percentile(non_zero, 99)
            data[data .> p99] .= p99
            data[(data .> 0) .& (data .< p1)] .= p1
        end
        
        core_raster_wins = Rasters.rebuild(core_raster, data)
        
        # Plot
        p_map = plot(core_raster_wins,
            title="SEZ Core Area - $year",
            color=:viridis,
            size=(800, 600))
        
        # Add Bokeo boundary
        if nrow(bokeo_province) > 0
            plot!(p_map, bokeo_province.geometry[1],
                fillalpha=0,
                linewidth=2,
                linecolor=:white,
                label="Bokeo")
        end
        
        savefig(p_map, joinpath(OUTPUT_DIR, "plots", "sez_core_$year.png"))
    end
end

# Calculate growth rates
growth_rates = DataFrame()

for i in 2:nrow(sez_core_data)
    prev = sez_core_data[i-1, :]
    curr = sez_core_data[i, :]
    
    bokeo_growth = (curr.bokeo_core_light / prev.bokeo_core_light - 1) * 100
    non_bokeo_growth = (curr.non_bokeo_core_light / prev.non_bokeo_core_light - 1) * 100
    
    push!(growth_rates, (
        year = curr.year,
        bokeo_growth = bokeo_growth,
        non_bokeo_growth = non_bokeo_growth
    ))
end

# Plot growth rates
p_growth = plot(growth_rates.year, growth_rates.bokeo_growth,
    label="Bokeo Growth",
    marker=:circle,
    linewidth=2,
    color=:red,
    title="Year-over-Year Growth Rates",
    xlabel="Year",
    ylabel="Growth Rate (%)",
    size=(800, 600))

plot!(p_growth, growth_rates.year, growth_rates.non_bokeo_growth,
    label="Non-Bokeo Growth",
    marker=:square,
    linewidth=2,
    color=:blue)

hline!(p_growth, [0], color=:black, linestyle=:dash, alpha=0.5, label="")

savefig(p_growth, joinpath(OUTPUT_DIR, "plots", "sez_growth_rates.png"))

# Save data
CSV.write(joinpath(OUTPUT_DIR, "data", "sez_core_comparison.csv"), sez_core_data)
CSV.write(joinpath(OUTPUT_DIR, "data", "sez_growth_rates.csv"), growth_rates)

# Summary statistics
avg_bokeo_pct = mean(sez_core_data.bokeo_percentage)
avg_bokeo_growth = mean(growth_rates.bokeo_growth)
avg_non_bokeo_growth = mean(growth_rates.non_bokeo_growth)

19.635902f0